In [37]:
from llama_index.core import VectorStoreIndex, Settings, SimpleDirectoryReader
from llama_index.core.evaluation import FaithfulnessEvaluator
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import ReActAgent

In [52]:
documents = SimpleDirectoryReader("../data").load_data()
llm = Ollama(model="llama3", request_timeout=120.0)


In [53]:
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

c:\Users\Enes227\environments\llama_environment\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [50]:
vector_index = VectorStoreIndex.from_documents(documents,embed_model=Settings.embed_model,show_progress=True)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/22 [00:00<?, ?it/s]

In [54]:
query_engine = vector_index.as_query_engine(llm=llm)
response = query_engine.query(
    "What does the author do to live?"
)
print(response)

Based on the provided context, it can be inferred that the author works multiple jobs or pursues various ventures to make a living. Initially, he is a student at the Accademia and later returns to RISD, but ultimately decides to get a job at Interleaf to fund his studies. After completing his education, he begins working on software for creating documents, which leads him to learn about low-end software eating high-end software. He also works on spam filters, paints still lives, and buys an office building in Cambridge. Additionally, he starts publishing essays online and becomes involved with startups, which ultimately leads him to meet Jessica Livingston and collaborate on a book of interviews with startup founders.


In [55]:
import nest_asyncio
nest_asyncio.apply()

In [61]:
evaluator = FaithfulnessEvaluator(llm=llm)
eval_result = evaluator.evaluate_response(response=response)
print(str(eval_result.passing))

False


In [34]:
from llama_index.core.llms import MockLLM
from llama_index.core import MockEmbedding

llm = MockLLM(max_tokens=256)
embed_model = MockEmbedding(embed_dim=1536)

In [62]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)
agent = ReActAgent.from_tools([multiply_tool], llm=llm, verbose=True)

In [63]:
agent.chat("What is 2123 * 215123")

Thought: I can use a tool to help me calculate the multiplication result.
Action: multiply
Action Input: {'a': 2123, 'b': 215123}
Observation: 456706129
Thought: The result of multiplying 2123 and 215123 is indeed 456706129.
Answer: 456706129


AgentChatResponse(response='456706129', sources=[ToolOutput(content='456706129', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 2123, 'b': 215123}}, raw_output=456706129, is_error=False)], source_nodes=[], is_dummy_stream=False)